In [1]:
import numpy as np
from numpy import exp,array,random,dot
import pandas as pd
from tqdm import *

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [2]:
def normalize(matrix):
    n = matrix.shape[1]
    norm_matrix = matrix

    for i in range(n):
        col_mean = np.mean(matrix[:,i])
        col_stdv = np.std(matrix[:,i])
    norm_matrix[:,i] = (norm_matrix[:,i] - col_mean) / col_stdv
    return norm_matrix

def normalize1(matrix):
    return (matrix - matrix.mean()) / matrix.std()

In [19]:
# Reading data from csv
data = pd.read_csv("data/train.csv")

In [4]:
# Classifier data entries to dummy coloumns
#data = pd.get_dummies(data)
#data.to_csv("data/data_afterdum.csv", sep=',',header=True,index=False)

In [20]:
# Spliting the data into Training data and Validation data
train, validate = np.split(data.sample(frac=1), [int(.7*len(data))])
print(len(train),len(validate))
#print(train)
#print(validate)

27281 11692


In [28]:
trn_ids = train[['id']]
#print(train_ids)
trn_y = train[['salary']]
#print(train_y)
trn_features = train.drop('id', axis=1)
trn_features = trn_features.drop('salary', axis=1)
trn_features = trn_features.drop('race', axis=1)
trn_features = trn_features.drop('native-country', axis=1)
trn_features = pd.get_dummies(trn_features)
trn_features = normalize1(trn_features)
print(trn_features.shape)

(27281, 61)


In [27]:
vld_ids = validate[['id']]
#print(valid_ids)
vld_y = validate[['salary']]
#print(train_y)
vld_features = validate.drop('id', axis=1)
vld_features = vld_features.drop('salary', axis=1)
print(trn_features.shape)

(27281, 61)


In [29]:
test_data = pd.read_csv("data/test.csv")
test_ids = test_data['id']
#print(test_ids)
tst_features = test_data.drop('id', axis=1)
tst_features = tst_features.drop('native-country', axis=1)
tst_features = tst_features.drop('race', axis=1)
tst_features = pd.get_dummies(tst_features)
tst_features = normalize1(tst_features)
print(tst_features.shape)

(6878, 61)


In [23]:
# Now Neural Network implementation starts

# Activation functions
def sigmoid(x):
    return 1 / ( 1 + np.exp(-x))

def derv_sigmoid(x):
    return x * (1 - x)

def tanh(x):
        return np.tanh(x)

def derv_tanh(x):
        return (1.0 - np.square(np.tanh(x)))

def accuracy(actual_results,expected_results):
    correct=0
    for i in range(0,len(result)):
        if result[i]==expected_results[i]:
            correct+=1
    return (correct/len(expected_results))

In [31]:
class NeuralNetwork():
    def __init__(self):
        random.seed(1)

        # setting the number of nodes in layer 2 and layer 3
        # more nodes --> more confidence in predictions (?)
        l1_nodes=61
        l2_nodes=60
        l3_nodes=60
        l4_nodes=1

        # assign random weights to matrices in network
        # format is (no. of nodes in previous layer) x (no. of nodes in following layer)
        self.weights1 = 2 * random.random((l1_nodes, l2_nodes)) -1
        self.weights2 = 2 * random.random((l2_nodes, l3_nodes)) -1
        self.weights3 = 2 * random.random((l3_nodes, l4_nodes)) -1

    # activation functions
    def tanh(self,x):
        return np.tanh(x)

    def derv_tanh(self,x):
        return (1.0 - (np.tanh(x))**2)

    # train neural network, adusting synaptic weights each time
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in tqdm(range(number_of_training_iterations)):
            eta=0.000001
            # print(iteration)

            # pass training set through our neural network i.e. feed fwd
            # a2 means the activations fed to second layer
            a2 = self.tanh(dot(training_set_inputs, self.weights1))
            a3 = self.tanh(dot(a2, self.weights2))
            output = self.tanh(dot(a3, self.weights3))

            # calculate 'error'
            del4 = (training_set_outputs - output)*self.derv_tanh(output)

            # find 'errors' in each layer
            del3 = dot(self.weights3, del4.T)*(self.derv_tanh(a3).T)
            del2 = dot(self.weights2, del3)*(self.derv_tanh(a2).T)

            # get adjustments (gradients) for each layer
            adjustment3 = eta*dot(a3.T, del4)
            adjustment2 = eta*dot(a2.T, del3.T)
            adjustment1 = eta*dot(training_set_inputs.T, del2.T)

            # adjust weights accordingly
            self.weights1 -= adjustment1
            self.weights2 -= adjustment2
            self.weights3 -= adjustment3
        return self.weights1,self.weights2,self.weights3


# initialise single neuron neural network
neural_network = NeuralNetwork()

print ("Random starting synaptic weights (layer 1): ")
print (neural_network.weights1)
print ("\nRandom starting synaptic weights (layer 2): ")
print (neural_network.weights2)
print ("\nRandom starting synaptic weights (layer 3): ")
print (neural_network.weights3)

# the training set.
training_set_inputs = np.array(trn_features)
training_set_outputs = np.array(trn_y.T)

fweights1,fweights2,fweights3=neural_network.train(training_set_inputs, training_set_outputs.T,100)

print ("\nNew synaptic weights (layer 1) after training: ")
print (neural_network.weights1)
print ("\nNew synaptic weights (layer 2) after training: ")
print (neural_network.weights2)
print ("\nNew synaptic weights (layer 3) after training: ")
print (neural_network.weights3)

# getting final results
after_l1 = tanh(np.dot(tst_features,fweights1))
after_l2 = tanh(np.dot(after_l1,fweights2))
after_l3 = tanh(np.dot(after_l2,fweights3))
print("after_l3")
print(after_l3)
predicted = []
after_l3 = after_l3.ravel()

result=np.column_stack((test_ids,after_l3))
np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')
print(after_l3)
for i in range(len(after_l3)):
    if after_l3[i] > 0:
        predicted.append(1)
    else:
        predicted.append(0)
print("predicted")
print(predicted)
result=np.column_stack((test_ids,predicted))

np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')


Random starting synaptic weights (layer 1): 
[[-0.16595599  0.44064899 -0.99977125 ..., -0.70654285  0.17861107
   0.39951672]
 [-0.79533114 -0.17188802  0.38880032 ...,  0.84604907  0.42304952
  -0.75145808]
 [-0.96023973 -0.94757803 -0.94338702 ..., -0.47016088 -0.86733033
  -0.2598316 ]
 ..., 
 [-0.70263027  0.119373   -0.01134214 ..., -0.58346861  0.95999843
  -0.86821246]
 [-0.5275445  -0.38220106  0.81900671 ..., -0.01455834  0.87305485
  -0.9998061 ]
 [-0.98086621 -0.49609715  0.01135293 ...,  0.46948164 -0.66436799
  -0.40282108]]

Random starting synaptic weights (layer 2): 
[[ 0.12544406  0.65289153 -0.59896739 ..., -0.95165266 -0.15919597
  -0.35262117]
 [ 0.23898025  0.71952613 -0.01471912 ...,  0.56950987 -0.23791125
   0.70480658]
 [ 0.39469308  0.4265198   0.64397729 ...,  0.51626457  0.7668073
  -0.26610352]
 ..., 
 [ 0.3406808   0.49013178 -0.15557569 ..., -0.77186015 -0.41258981
  -0.9824219 ]
 [-0.49347261  0.67542556  0.61551205 ..., -0.24817142 -0.06177893
   0.894

100%|████████████████████████████████████████| 100/100 [01:18<00:00,  1.30it/s]



New synaptic weights (layer 1) after training: 
[[-0.30869826  0.51920053 -0.80940782 ..., -0.8120994   0.14218819
   0.40142979]
 [-0.53461088 -0.25373344  0.15132065 ...,  1.00585221  0.46538036
  -0.76927207]
 [-0.68123625 -1.03439086 -1.21584235 ..., -0.29819446 -0.80039435
  -0.29532629]
 ..., 
 [-0.46730647  0.08701386 -0.20690197 ..., -0.46199499  0.97789207
  -0.87776247]
 [-0.07737694 -0.54723283  0.36334374 ...,  0.31800285  0.94720831
  -1.02380019]
 [-1.43103377 -0.33106538  0.46701589 ...,  0.13692045 -0.73852145
  -0.37882698]]

New synaptic weights (layer 2) after training: 
[[-0.04103334  0.48362741 -0.7335344  ..., -1.06395129 -0.27652619
  -0.32210802]
 [ 0.25479164  0.72688598 -0.01014593 ...,  0.57358058 -0.23285181
   0.7040734 ]
 [ 0.49052583  0.51599503  0.71484796 ...,  0.57662762  0.82881439
  -0.28134201]
 ..., 
 [ 0.25202875  0.41188244 -0.22018309 ..., -0.82270648 -0.46606659
  -0.9693907 ]
 [-0.57828564  0.58037422  0.53931088 ..., -0.30867097 -0.12221767


In [32]:
np.savetxt("data/output.csv",result,delimiter=',',fmt="%d,%f",header="id,salary",comments ='')